In [ ]:
import sys
sys.path.append('..')
from economic_brazil.coleta_dados.economic_data_brazil import EconomicBrazil
from economic_brazil.coleta_dados.tratando_economic_brazil import read_indice_abcr,sondagem_industria
from economic_brazil.processando_dados.tratando_dados import TratandoDados
from economic_brazil.treinamento.treinamento_algoritimos import TreinandoModelos
import warnings
import pandas as pd
import pickle
import os

path_codigos_rodando = os.path.join(os.getcwd())

warnings.filterwarnings("ignore", category=UserWarning)

banco_central_codes = {
    "indice_condicoes_economicas": 4394,
    "indice_valores_garantias_imoveis_residencias_financiados": 21340,
    "venda_veiculos_concessionarias": 1378,
    "indicador_movimento_comercio_prazo": 1453,
    "indice_volume_vendas_varejo": 1455,
    "imposto_sobre_produtos": 22098,
    'metas_inflacao': 13521,
    'indice_expectativas_futuras': 4395,
    'indice_confianca_empresarial_industrial': 7343,
    'selic': 4189,
    'ibc':24364,
    'Producao_de_veiculos':1373,
    'Producao_de_aco':7357,
    

}

variaveis_ibge = {
    "ipca": {
        "codigo": '1737',
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "63",
    },
    "custo_m2": {
        "codigo": '2296',
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "1198",
    },
    "pmc_volume": {
        "codigo": '8186',
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "11709",
    },
    "producao_fisica_industrial": {
        "codigo": '8159',
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "11599",},

    "producao_fisica_para_construcao_civil": {
        "codigo": '7980',
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "11599",},
    
    "producao_soja_milho": {    
        "codigo": '6588',
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "35",},
    
    "precos_construcao_civil": {
        "codigo": '2296',
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "1198",},
    
    "volume_servicos_(pms)": {
        "codigo": '8162',
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "11621",
    }, 
    
    "taxa_desocupacao": { 
        "codigo": '6381',
        "territorial_level": "1",
        "ibge_territorial_code": "all",
        "variable": "4099",
            
            }
}

codigos_ipeadata_padrao = {
    "rendimento_real_medio": "PNADC12_RRTH12",
    "pessoas_forca_trabalho": "PNADC12_FT12",
    "caged_novo": "CAGED12_SALDON12",
    "caged_antigo": "CAGED12_SALDO12",
    'ibovespa': 'ANBIMA12_IBVSP12',
    "taja_juros_ltn": "ANBIMA12_TJTLN1212",
    'exportacoes': 'PAN12_XTV12',
    'importacoes': 'PAN12_MTV12',
    'm_1': 'BM12_M1MN12',
    'taxa_cambio': 'PAN12_ERV12',
    'atividade_economica': 'SGS12_IBCBR12',
    'producao_industrial': 'PAN12_QIIGG12',
    'producao_industrial_intermediario': 'PIMPFN12_QIBIN12',
    'capcidade_instalada': 'CNI12_NUCAP12',
    'caixas_papelao': 'ABPO12_PAPEL12',
    'faturamento_industrial': 'CNI12_VENREA12',
    'importacoes_industrial': 'FUNCEX12_MDQT12',
    'importacoes_intermediario': 'FUNCEX12_MDQBIGCE12',
    'confianca_empresario_exportador': 'CNI12_ICEIEXP12',
    'confianca_empresario_atual': 'CNI12_ICEICA12',
    'confianca_consumidor':'FCESP12_IIC12',
    'ettj_26': 'ANBIMA366_TJTLN6366',    
}

lista = ['seguro desemprego', 
         'empregos',
         'FGTS',
         'INSS']

indicadores_ibge_link = {
    "pib": "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela5932.xlsx&terr=N&rank=-&query=t/5932/n1/all/v/6564/p/all/c11255/90707/d/v6564%201/l/v,p,t%2Bc11255&verUFs=false&verComplementos2=false&verComplementos1=false&omitirIndentacao=false&abreviarRotulos=false&exibirNotas=false&agruparNoCabecalho=false",
    "despesas_publica": "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela5932.xlsx&terr=N&rank=-&query=t/5932/n1/all/v/6561/p/all/c11255/93405/d/v6561%201/l/v,p%2Bc11255,t",
    "capital_fixo": "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela5932.xlsx&terr=N&rank=-&query=t/5932/n1/all/v/6561/p/all/c11255/93406/d/v6561%201/l/v,p%2Bc11255,t",
    "producao_industrial_manufatureira": "https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela8158.xlsx&terr=N&rank=-&query=t/8158/n1/all/v/11599/p/all/c543/129278/d/v11599%205/l/v,p%2Bc543,t",
    'soja' :'https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela6588.xlsx&terr=N&rank=-&query=t/6588/n1/all/v/35/p/all/c48/0,39443/l/v,p%2Bc48,t',
    'milho_1' :'https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela6588.xlsx&terr=N&rank=-&query=t/6588/n1/all/v/35/p/all/c48/0,39441/l/v,p%2Bc48,t',
    'milho_2' :'https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela6588.xlsx&terr=N&rank=-&query=t/6588/n1/all/v/35/p/all/c48/0,39442/l/v,p%2Bc48,t',
    'pms': 'https://sidra.ibge.gov.br/geratabela?format=xlsx&name=tabela8162.xlsx&terr=N&rank=-&query=t/8162/n1/all/v/11622/p/all/c11046/56726/c12355/107071/d/v11622%205/l/v,p%2Bc11046,t%2Bc12355'
}

codigos_fred = {'nasdaq100':'NASDAQ100',
                'taxa_cambio_efetiva':'RBBRBIS',
                'cboe_nasdaq':'VXNCLS',
                'taxa_juros_interbancaria':'IRSTCI01BRM156N',
                'atividade_economica_eua':'USPHCI',
                'indice_confianca_manufatura':'BSCICP03BRM665S',
                'indice_confianca_exportadores':'BSXRLV02BRM086S',
                'indice_tendencia_emprego':'BRABREMFT02STSAM',
                'indice_confianca_consumidor':'CSCICP03BRM665S',
                'capacidade_instalada':'BSCURT02BRM160S',}

############################################################################## VARIAVEL PREDICAO ################################################################
variavel_predicao = 'producao_industrial'


################################################################################ DADOS ############################################################################

dados_bcb = True
dados_ibge = True
dados_expectativas_inflacao = False
dados_metas_inflacao = False
dados_ibge_link = True
dados_ipeadata = True
dados_google_trends = True
dados_fred =True
data_inicio = '2003-01-01'

economic_brasil = EconomicBrazil(codigos_banco_central=banco_central_codes, 
                       codigos_ibge=variaveis_ibge,
                       codigos_ipeadata=codigos_ipeadata_padrao,
                       lista_termos_google_trends=lista,
                       codigos_ibge_link=indicadores_ibge_link,
                       codigos_fred=codigos_fred,
                       data_inicio=data_inicio)

dados = economic_brasil.dados_brazil(
                dados_bcb=dados_bcb,
                dados_ipeadata=dados_ipeadata,
                dados_fred=dados_fred,
                dados_google_trends=dados_google_trends,
                dados_expectativas_inflacao=dados_expectativas_inflacao,
                dados_ibge_link=dados_ibge_link,
                dados_ibge_codigos=dados_ibge,
    
)
#Indice ibrc
try:
    dados_ibcr = read_indice_abcr()
    dados = dados.join(dados_ibcr).ffill()
except:
    print('Nao foi possivel carregar indice ibcr')
    
try:
    dados = dados.join(sondagem_industria(4, "sond_estoques"))
    dados = dados.join(sondagem_industria(6, "sond_demanda"))
    dados = dados.join(sondagem_industria(7, "sond_exportacao"))
    dados = dados.join(sondagem_industria(8, "sond_compras"))
    dados = dados.join(sondagem_industria(9, "sond_empregados"))
    dados = dados.ffill().bfill()
except:
    print('Nao foi possivel carregar sondagem da industrial')


########################################################################### TRATAMENTO ############################################################################

tratando_scaler = True
tratando_pca = False
tratando_dummy_covid = True
tratando_defasagens = True
tratando_datas = True
tratando_estacionaridade = True
tratando_rfe = True
tratando_smart_correlation = True
tratando_variance = True
tratando_estacionaridade_log = True
data_divisao = '2020-06-01'

tratando = TratandoDados(dados,
                         coluna_label=variavel_predicao,
                         data_divisao=data_divisao,
                         numero_defasagens=4,
                         n_features_to_select=100,
                         scaler=tratando_scaler, 
                        pca=tratando_pca, 
                        covid=tratando_dummy_covid, 
                        datas=tratando_datas, 
                        defasagens=tratando_defasagens, 
                        estacionaridade=tratando_estacionaridade,
                        rfe=tratando_rfe,
                        smart_correlation=tratando_smart_correlation,
                        variancia=tratando_variance,
                        estacionaridade_log=tratando_estacionaridade_log)

x_treino, x_teste, y_treino, y_teste, pca , scaler , rfe_model, variance_model, smart_model = tratando.tratando_dados()
         

In [10]:
data_divisao_treino_teste = tratando.data_divisao_treino_teste()

param_grid_gradiente = {
                    "n_estimators": [2, 10,20,30, 50, 100, 200, 300, 500],
                    "learning_rate": [0.1, 0.15, 0.2, 0.4, 0.6, 0.8, 1, 1.25, 1.5,1.75],
                    "max_depth": [1, 3, 5, 7, 9],
                }

param_grid_xgboost = {
                    "n_estimators": [2, 10,20,30, 50, 100, 200, 300, 500],
                    "learning_rate": [0.1, 0.15, 0.2, 0.4, 0.6, 0.8, 1, 1.25, 1.5,1.75],
                    "max_depth": [1, 3, 5, 7, 9],
                }

param_grid_catboost = {
                    "iterations": [2, 10,20,30, 50, 100, 200, 300, 500],
                    "learning_rate": [0.1, 0.15, 0.2, 0.4, 0.6, 0.8, 1],
                    "depth": [1, 3, 5, 7, 9,10,15],
                }


tuning = TreinandoModelos(x_treino, 
                          y_treino, 
                          x_teste, 
                          y_teste,
                          numero_divisoes = 5,
                          diretorio=f'../codigos_rodando/modelos_salvos/modelos_{variavel_predicao}/',
                          salvar_modelo=True)

modelo_redes_neurais = True
modelo_cast = True
modelo_sarimax = False
modelo_gradient_boosting = True
modelo_regresao_linear = True
modelo_xgboost = True
redes_neurais_tuning = False

modelos_tunning = tuning.treinar_modelos(redes_neurais=modelo_redes_neurais,
                                         cat_boost=modelo_cast,
                                         sarimax=modelo_sarimax,
                                         gradiente_boosting=modelo_gradient_boosting,
                                         regressao_linear=modelo_regresao_linear,
                                         xg_boost=modelo_xgboost,
                                         redes_neurais_tuning=redes_neurais_tuning,
                                         param_grid_catboost=param_grid_catboost,
                                        param_grid_xgboost=param_grid_xgboost,
                                        param_grid_gradiente=param_grid_gradiente,
                                         )

dados_salvos = {}
dados_salvos['dados'] = dados
dados_salvos['x_treino'] = x_treino
dados_salvos['x_teste'] = x_teste
dados_salvos['y_treino'] = y_treino
dados_salvos['y_teste'] = y_teste
if pca:
    dados_salvos['pca'] = pca
if scaler:
    dados_salvos['scaler'] = scaler
if rfe_model:
    dados_salvos['rfe_model'] = rfe_model
if variance_model:
    dados_salvos['variance_model'] = variance_model
if smart_model:
    dados_salvos['smart_model'] = smart_model
dados_salvos['data_divisao_treino_teste'] = data_divisao_treino_teste
dados_salvos['tratando'] = tratando

with open(path_codigos_rodando+f'/avaliacao_modelos/apresentacao_streamlit/dados_treinamento_{variavel_predicao}.pkl', 'wb') as f:
    pickle.dump(dados_salvos, f)

Data divisao de treino e teste: 2020-06-01
Best Bayesian Optimization Params: OrderedDict([('learning_rate', 0.2), ('max_depth', 1), ('n_estimators', 200)]) Score: -3.439174570155246
Modelo Gradiente Boosting Treinado
Best Bayesian Optimization Params: OrderedDict([('learning_rate', 0.1), ('max_depth', 1), ('n_estimators', 500)]) Score: -3.4270194575275825
Modelo XG Boost Treinado
0:	learn: 7.5600684	total: 182ms	remaining: 36.2s
1:	learn: 7.2855884	total: 185ms	remaining: 18.3s
2:	learn: 7.0957587	total: 186ms	remaining: 12.2s
3:	learn: 6.8136752	total: 187ms	remaining: 9.18s
4:	learn: 6.5950972	total: 189ms	remaining: 7.36s
5:	learn: 6.4009724	total: 190ms	remaining: 6.14s
6:	learn: 6.1766190	total: 191ms	remaining: 5.27s
7:	learn: 6.0680978	total: 192ms	remaining: 4.62s
8:	learn: 5.8322445	total: 193ms	remaining: 4.1s
9:	learn: 5.6764561	total: 195ms	remaining: 3.7s
10:	learn: 5.4323211	total: 196ms	remaining: 3.37s
11:	learn: 5.2443205	total: 197ms	remaining: 3.09s
12:	learn: 5.087

In [11]:
import sys
sys.path.append('..')
from economic_brazil.treinamento.treinamento_algoritimos import carregar
from economic_brazil.analisando_modelos.analise_modelos_regressao import MetricasModelosDicionario,PredicaosModelos
from economic_brazil.analisando_modelos.regressao_conformal import ConformalRegressionPlotter
from economic_brazil.predicao_valores_futuros import KerasTrainedRegressor, Predicao
import pandas as pd
from joblib import parallel_config
import warnings
import pickle
import os
warnings.filterwarnings("ignore", category=UserWarning)
path_codigos_rodando = os.path.join(os.getcwd())

##########################################################################################################DEFININDO VARIAVEIS############################################################################################################

variavel_predicao = 'producao_industrial'

##########################################################################################CARREGANDO DADOS############################################################################################################

arquivo_pib = path_codigos_rodando+f'/avaliacao_modelos/apresentacao_streamlit/dados_treinamento_{variavel_predicao}.pkl'
dados_carregados = pickle.load(open(arquivo_pib, 'rb'))

dados = dados_carregados['dados']
x_treino = dados_carregados['x_treino']
x_teste = dados_carregados['x_teste']
y_treino = dados_carregados['y_treino']
y_teste = dados_carregados['y_teste']
if 'pca' in dados_carregados.keys():
    pca = dados_carregados['pca']
if 'scaler' in dados_carregados.keys():    
    scaler = dados_carregados['scaler']
if 'rfe_model' in dados_carregados.keys():
    rfe_model = dados_carregados['rfe_model']
if 'smart_model' in dados_carregados.keys():
    smart_model = dados_carregados['smart_model']
if 'variance_model' in dados_carregados.keys():
    variance_model = dados_carregados['variance_model']
data_divisao_treino_teste = dados_carregados['data_divisao_treino_teste']
tratando = dados_carregados['tratando']

##########################################################################################################CARREGANDO MODELOS############################################################################################################

modelos_carregados = carregar(diretorio=path_codigos_rodando+f'/modelos_salvos/modelos_{variavel_predicao}/',gradiente_boosting=True, xg_boost=True, cat_boost=True, regressao_linear=True, redes_neurais=True, sarimax=False)


##########################################################################################################PREVENDO OS DADOS DE TREINO E TESTE################################################################################
predi = PredicaosModelos(modelos_carregados, x_treino, y_treino, x_teste, y_teste)
predicoes_treino, predicoes_teste = predi.predicoes()
x_treino, y_treino, x_teste, y_teste, x_treino_recorrente, y_treino_recorrente, x_teste_recorrente, y_teste_recorrente = predi.return_dados()

###########################################################################################################################CALCULANDO METRICAS################################################################################
metri = MetricasModelosDicionario()
metrica_teste = metri.calculando_metricas(predicoes_teste, y_teste, y_teste_recorrente)
metrica_treino = metri.calculando_metricas(predicoes_treino, y_treino, y_treino_recorrente)

print(metrica_teste)
print(metrica_treino)

#metrica_teste.to_csv("/workspaces/Predicoes_macroeconomicas/codigos_rodando/avaliacao_modelos/metricas_teste.csv")
#metrica_treino.to_csv("/workspaces/Predicoes_macroeconomicas/codigos_rodando/avaliacao_modelos/metricas_treino.csv")

index_treino = dados[dados.index <= data_divisao_treino_teste][6:].index
index_teste = dados[dados.index > data_divisao_treino_teste][4:].index

#############################################################################################################PLOTANDO METRICAS################################################################################

################################################################################################AVALIANDO MODELOS################################################################################
modelos_validos_teste = metrica_teste[metrica_teste['Variance'] > 0.05]

if modelos_validos_teste.empty:
    # Calcular a diferença absoluta entre métricas de treino e teste
    metrica_treino = metrica_treino.reindex(metrica_teste.index)
    metricas_diferenca = pd.DataFrame(index=metrica_treino.index)
    metricas_diferenca['MAE_diff'] =abs(metrica_treino['MAE'] - metrica_teste['MAE'])
    metricas_diferenca['MSE_diff'] = abs(metrica_treino['MSE'] - metrica_teste['MSE'])
    metricas_diferenca['RMSE_diff'] = abs(metrica_treino['RMSE'] - metrica_teste['RMSE'])

    # Calcular a soma das diferenças
    metricas_diferenca['total_diff'] = (metricas_diferenca['MAE_diff'] +
                                        metricas_diferenca['MSE_diff'] +
                                        metricas_diferenca['RMSE_diff'])
    # Selecionar o modelo com a menor soma das diferenças
    melhor_modelo = metricas_diferenca['total_diff'].idxmin()
    menor_diferenca = metricas_diferenca['total_diff'].min()

    print(f'Melhor modelo baseado na menor diferença entre treino e teste, com valor de {menor_diferenca}:', melhor_modelo)
    print(metricas_diferenca)
else:
    melhor_modelo = modelos_validos_teste['MAE'].idxmin()
    print(f'Melhor modelo baseado na MAE mais baixo, com valor de {metrica_teste["MAE"].min()}:',melhor_modelo)
    
#Conformal
index_treino_conformal = dados[dados.index <= data_divisao_treino_teste].index
index_teste_conformal = dados[dados.index > data_divisao_treino_teste].index

with parallel_config(backend='threading', n_jobs=2):
    if melhor_modelo == 'redes_neurais':
        conformal = ConformalRegressionPlotter(KerasTrainedRegressor(modelos_carregados[melhor_modelo]), x_treino_recorrente, x_teste_recorrente, y_treino[1:], y_teste[1:])
        y_pred, y_pis, _,_ = conformal.regressao_conformal()
        conformal.plot_prediction_intervals(index_train=index_treino_conformal, index_test=index_teste_conformal,title=f'Predição Intervals {melhor_modelo}',save=True,diretorio=path_codigos_rodando+f'/avaliacao_modelos/regressao_conforma_teste_{variavel_predicao}.png')
    else:
        conformal = ConformalRegressionPlotter(modelos_carregados[melhor_modelo], x_treino, x_teste, y_treino, y_teste)
        y_pred, y_pis, _,_ = conformal.regressao_conformal()
        conformal.plot_prediction_intervals(index_train=index_treino_conformal, index_test=index_teste_conformal,title=f'Predição Intervals {melhor_modelo}',save=True,diretorio=path_codigos_rodando+f'/avaliacao_modelos/regressao_conforma_teste_{variavel_predicao}.png')

dados_corformal = {
    'data': index_teste[-1],
    'predicao': y_pred,
    'intervalo_lower': y_pis.squeeze()[0:,0],
    'intervalo_upper': y_pis.squeeze()[0:,1],}    
## Predicao futuro
with parallel_config(backend='threading', n_jobs=2):
    predicao = Predicao(x_treino,y_treino,tratando,dados,melhor_modelo,modelos_carregados[melhor_modelo],periodo='Trimestral',coluna=variavel_predicao)
    
dados_predicao_futuro, _, index_futuro = predicao.criando_dados_futuros()
dados_predicao = predicao.criando_dataframe_predicoes()
#dados_predicao.to_csv('/workspaces/Predicoes_macroeconomicas/codigos_rodando/avaliacao_modelos/dados_predicao.csv',index=False)
print(dados_predicao)
data_predicao,intervalo_lower,intervalo_upper,predicao_proximo_mes = predicao.predicao_ultimo_periodo()
dados_futuro = {
    'data': data_predicao,
    'intervalo_lower': intervalo_lower,
    'intervalo_upper': intervalo_upper,
    'predicao': predicao_proximo_mes}
print(f'Data da predição:{data_predicao}, Valor da predição:{predicao_proximo_mes}, Intervalo de predição [lower,upper] :{intervalo_lower,intervalo_upper}')
predicao.plotando_predicoes(save=True,diretorio=path_codigos_rodando+f'/avaliacao_modelos/predicao_futuro_{variavel_predicao}.png')

###salvando os dados
dados_salvos = {}
dados_salvos['dados'] = dados
dados_salvos['y_treino'] = y_treino
dados_salvos['x_treino'] = x_treino
dados_salvos['y_treino_recorrente'] = y_treino_recorrente
dados_salvos['x_treino_recorrente'] = x_treino_recorrente
dados_salvos['x_teste'] = x_teste
dados_salvos['y_teste'] = y_teste
dados_salvos['y_teste_recorrente'] = y_teste_recorrente
dados_salvos['x_teste_recorrente'] = x_teste_recorrente
dados_salvos['metrica_teste'] = metrica_teste
dados_salvos['metrica_treino'] = metrica_treino
dados_salvos['predicoes_treino'] = predicoes_treino
dados_salvos['predicoes_teste'] = predicoes_teste
dados_salvos['index_treino'] = index_treino
dados_salvos['index_teste'] = index_teste
dados_salvos['melhor_modelo'] = melhor_modelo
dados_salvos['dados_conformal'] = dados_corformal
dados_salvos['dados_predicao'] = dados_predicao
dados_salvos['dados_futuro'] = dados_futuro
dados_salvos['modelos_carregados'] = list(modelos_carregados.keys())

with open(path_codigos_rodando+f'/avaliacao_modelos/apresentacao_streamlit/dados_salvos_{variavel_predicao}.pkl', 'wb') as f:
    pickle.dump(dados_salvos, f)
    
print('Avaliacao concluida com sucesso!')


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
                        MAE         MSE     RMSE        R²  Variance
cat_boost              4.02       53.31     7.30     -0.03      0.77
gradiente_boosting     6.90       91.03     9.54     -0.76     17.82
xg_boost               6.95       91.00     9.54     -0.76     14.00
redes_neurais         16.95      312.19    17.67     -4.92      5.61
regressao_linear    1305.33  1720315.73  1311.61 -33354.08  16481.42
                     MAE   MSE  RMSE    R²  Variance
cat_boost           0.03  0.00  0.03  1.00     48.46
redes_neurais       0.31  0.19  0.44  1.00     47.32
xg_boost            1.11  1.91  1.38  0.96     42.25
gradiente_boosting  1.15  2.02  1.42  0.96     42.22
regressao_linear    1.18  2.40  1.55  0.95     46.23
Melhor modelo baseado na MAE mais baixo, com valor de 4.02: cat_boost
0:	learn: 6.6268191	total: 19.2ms	remaining: 3.82s
1:	learn: 6.3717321	total: 23.4ms	remaining: 2.32s
2:	learn: 6.0890367	

In [18]:
import os
path_diretorio = os.getcwd()+'/avaliacao_modelos/apresentacao_streamlit'
arquivo_selic = path_diretorio+'/dados_salvos_selic.pkl'
dados_salvos_selic = pickle.load(open(arquivo_selic, 'rb'))
arquivo_ipca = path_diretorio+'/dados_salvos_ipca.pkl'
dados_salvos_ipca = pickle.load(open(arquivo_ipca, 'rb'))
arquivo_pib = path_diretorio+'/dados_salvos_pib.pkl'
dados_salvos_pib = pickle.load(open(arquivo_pib, 'rb'))
arquivo_taxa_desocupacao = path_diretorio+'/dados_salvos_taxa_desocupacao.pkl'
dados_salvos_taxa_desocupacao = pickle.load(open(arquivo_taxa_desocupacao, 'rb'))
arquivo_producao_industrial = path_diretorio+'/dados_salvos_producao_industrial.pkl'
dados_salvos_producao_industrial = pickle.load(open(arquivo_producao_industrial, 'rb'))

In [19]:
dados_futuros_selic = dados_salvos_selic['dados_futuro']
dados_futuros_ipca = dados_salvos_ipca['dados_futuro']
dados_futuros_pib = dados_salvos_pib['dados_futuro']
dados_futuros_taxa_desocupacao = dados_salvos_taxa_desocupacao['dados_futuro']
dados_futuros_producao_industrial = dados_salvos_producao_industrial['dados_futuro']

In [28]:
def juntar_dados(primeira_vez=True,recebe_data=None,dicionario_1=None, dicionario_2=None,variavel_1=None,variavel_2=None):
    if primeira_vez:
        data_1 = pd.DataFrame([dicionario_1])
        data_1['Variavel'] = variavel_1
        data_2 = pd.DataFrame([dicionario_2])
        data_2['Variavel'] = variavel_2
        return pd.concat([data_1,data_2])
    else:
        data_1 = pd.DataFrame([dicionario_1])
        data_1['Variavel'] = variavel_1
        return pd.concat([data_1,recebe_data])
    
predicoes = juntar_dados(primeira_vez=True,recebe_data=None,dicionario_1=dados_futuros_selic, dicionario_2=dados_futuros_ipca,variavel_1='selic',variavel_2='ipca')
predicoes = juntar_dados(primeira_vez=False,recebe_data=predicoes,dicionario_1=dados_futuros_pib,variavel_1='pib')
predicoes = juntar_dados(primeira_vez=False,recebe_data=predicoes,dicionario_1=dados_futuros_taxa_desocupacao,variavel_1='taxa_desocupacao')
predicoes = juntar_dados(primeira_vez=False,recebe_data=predicoes,dicionario_1=dados_futuros_producao_industrial,variavel_1='producao_industrial')
predicoes.index = predicoes['Variavel']
data = predicoes['data'].iloc[0]
predicoes.drop(['Variavel'],axis=1,inplace=True)
#predicoes = predicoes[['predicao', 'intervalo_upper', 'intervalo_lower']]

In [31]:
predicoes['data']['producao_industrial']

'2024-10-01'

In [24]:
dados_futuros_ipca

{'data': '2024-08-01',
 'intervalo_lower': -0.39,
 'intervalo_upper': 1.03,
 'predicao': 0.33}